# Complete Organizational Assessment Pipeline

This notebook demonstrates the complete assessment workflow from item-level scoring to organizational maturity classification.

## Pipeline Steps
1. Load assessment data
2. Compute item scores (ADLI + LeTCI)
3. Aggregate category scores
4. Calculate organizational score
5. Classify maturity level
6. Generate comprehensive report

In [ ]:
import sys
sys.path.append('../src')

from adli_letci_core import (
    AssessmentEngine,
    ADLIIndicators,
    LeTCIIndicators,
    compute_category_score,
    compute_organizational_score,
    classify_maturity_level
)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

sns.set_style('whitegrid')

print("Assessment engine loaded successfully!")

## Load Assessment Data

In [ ]:
# Load sample data
df = pd.read_csv('../data/examples/sample_assessment_data.csv')

print(f"Total assessments: {len(df)}")
print(f"Departments: {df['department'].nunique()}")
print(f"Process items: {len(df[df['item_type']=='Process'])}")
print(f"Results items: {len(df[df['item_type']=='Results'])}")

df.head()

## Complete Assessment for Single Department

In [ ]:
def assess_department(df, department_name):
    """
    Run complete assessment pipeline for a department.
    """
    dept_df = df[df['department'] == department_name]
    
    # Prepare process items
    process_items = []
    for _, row in dept_df[dept_df['item_type'] == 'Process'].iterrows():
        process_items.append({
            'item_id': row['item_id'],
            'category': row['category'],
            'adli': ADLIIndicators(
                approach=row['approach'],
                deployment=row['deployment'],
                learning=row['learning'],
                integration=row['integration']
            ),
            'point_value': row['point_value'],
            'deployment_gap': 1.0 - row['deployment']  # Estimate urgency from deployment
        })
    
    # Prepare results items
    results_items = []
    for _, row in dept_df[dept_df['item_type'] == 'Results'].iterrows():
        results_items.append({
            'item_id': row['item_id'],
            'category': row['category'],
            'letci': LeTCIIndicators(
                level=row['level'],
                trend=row['trend'],
                comparison=row['comparison'],
                integration=row['integration']
            ),
            'point_value': row['point_value'],
            'deployment_gap': 1.0 - row['level']  # Estimate from level
        })
    
    # Run assessment
    engine = AssessmentEngine()
    results = engine.compute_organizational_assessment(
        process_items=process_items,
        results_items=results_items,
        category_point_allocations={}
    )
    
    return results

# Assess Computer Science department
cs_assessment = assess_department(df, 'Computer Science')

print("Computer Science - Assessment Results")
print("=" * 50)
print(f"\nOrganizational Score: {cs_assessment['organizational_score']:.2f}/100")
print(f"Integration Health Index: {cs_assessment['ihi']:.3f}")
print(f"\nMaturity Classification:")
print(f"  Level: {cs_assessment['maturity_level']['level']}")
print(f"  Label: {cs_assessment['maturity_level']['label']}")
print(f"  Description: {cs_assessment['maturity_level']['description']}")
print(f"\nCategory Scores:")
for category, score in sorted(cs_assessment['category_scores'].items(), key=lambda x: x[1], reverse=True):
    print(f"  {category:15}: {score:5.2f}")

## Category Score Breakdown Visualization

In [ ]:
# Visualize category scores
categories = list(cs_assessment['category_scores'].keys())
scores = list(cs_assessment['category_scores'].values())

# Sort by Baldrige order
baldrige_order = ['Leadership', 'Strategy', 'Customers', 'Measurement', 'Workforce', 'Operations', 'Results']
ordered_scores = [cs_assessment['category_scores'][cat] for cat in baldrige_order]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bar chart
colors_cat = ['#e74c3c' if s < 70 else '#f39c12' if s < 85 else '#27ae60' for s in ordered_scores]
ax1.barh(baldrige_order, ordered_scores, color=colors_cat)
ax1.set_xlabel('Category Score', fontsize=12)
ax1.set_ylabel('Baldrige Category', fontsize=12)
ax1.set_title('Computer Science - Category Scores', fontsize=14, fontweight='bold')
ax1.set_xlim(0, 100)
ax1.axvline(70, color='orange', linestyle='--', alpha=0.5, label='Level 4 Threshold')
ax1.axvline(85, color='green', linestyle='--', alpha=0.5, label='Level 5 Threshold')
ax1.legend(fontsize=10)
ax1.grid(axis='x', alpha=0.3)

# Pie chart with category weights
weights = [0.12, 0.085, 0.085, 0.10, 0.10, 0.15, 0.36]
contributions = [s * w for s, w in zip(ordered_scores, weights)]
explode = [0.05 if cat == 'Results' else 0 for cat in baldrige_order]

ax2.pie(contributions, labels=baldrige_order, autopct='%1.1f%%', 
       explode=explode, shadow=True, startangle=90)
ax2.set_title('Contribution to Organizational Score\n(Score × Weight)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## Multi-Department Comparison

In [ ]:
# Assess all departments
departments = df['department'].unique()
all_assessments = {}

for dept in departments:
    all_assessments[dept] = assess_department(df, dept)

# Create comparison dataframe
comparison_data = []
for dept, assessment in all_assessments.items():
    comparison_data.append({
        'Department': dept,
        'Org Score': assessment['organizational_score'],
        'IHI': assessment['ihi'],
        'Maturity Level': assessment['maturity_level']['level'],
        'Maturity Label': assessment['maturity_level']['label']
    })

comparison_df = pd.DataFrame(comparison_data).sort_values('Org Score', ascending=False)

print("\nDepartment Comparison:")
print(comparison_df.to_string(index=False))

## Maturity Distribution Visualization

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Score distribution
maturity_colors = {1: '#e74c3c', 2: '#e67e22', 3: '#f39c12', 4: '#27ae60', 5: '#2ecc71'}
colors_list = [maturity_colors[level] for level in comparison_df['Maturity Level']]

ax1.barh(comparison_df['Department'], comparison_df['Org Score'], color=colors_list)
ax1.set_xlabel('Organizational Score', fontsize=12)
ax1.set_ylabel('Department', fontsize=12)
ax1.set_title('Organizational Scores by Department', fontsize=14, fontweight='bold')
ax1.set_xlim(0, 100)

# Add maturity band shading
ax1.axvspan(0, 20, alpha=0.1, color='red', label='Level 1')
ax1.axvspan(21, 40, alpha=0.1, color='orange')
ax1.axvspan(41, 60, alpha=0.1, color='yellow')
ax1.axvspan(61, 85, alpha=0.1, color='lightgreen')
ax1.axvspan(86, 100, alpha=0.1, color='green', label='Level 5')
ax1.grid(axis='x', alpha=0.3)

# IHI vs Score scatter
for idx, row in comparison_df.iterrows():
    ax2.scatter(row['IHI'], row['Org Score'], 
               s=200, alpha=0.6, 
               color=maturity_colors[row['Maturity Level']],
               edgecolors='black', linewidth=2)
    ax2.annotate(row['Department'], (row['IHI'], row['Org Score']),
                fontsize=9, ha='center', va='bottom')

ax2.set_xlabel('Integration Health Index', fontsize=12)
ax2.set_ylabel('Organizational Score', fontsize=12)
ax2.set_title('IHI vs Organizational Performance', fontsize=14, fontweight='bold')
ax2.grid(alpha=0.3)
ax2.set_xlim(0, 1.0)
ax2.set_ylim(0, 100)

plt.tight_layout()
plt.show()

## Generate Assessment Report

In [ ]:
def generate_assessment_report(department, assessment):
    """
    Generate formatted assessment report.
    """
    report = f"""
{'='*70}
ORGANIZATIONAL EXCELLENCE ASSESSMENT REPORT
{'='*70}

Department: {department}
Assessment Date: 2024-02-01

OVERALL PERFORMANCE
{'-'*70}
Organizational Score:        {assessment['organizational_score']:6.2f}/100
Integration Health Index:    {assessment['ihi']:6.3f}

MATURITY CLASSIFICATION
{'-'*70}
Maturity Level:              {assessment['maturity_level']['level']} - {assessment['maturity_level']['label']}
Description:                 {assessment['maturity_level']['description']}
Score Range:                 {assessment['maturity_level']['range'][0]}-{assessment['maturity_level']['range'][1]}

CATEGORY BREAKDOWN
{'-'*70}
"""
    
    for category in baldrige_order:
        score = assessment['category_scores'][category]
        report += f"{category:15}: {score:5.2f}\n"
    
    report += f"""
TOP 5 IMPROVEMENT PRIORITIES
{'-'*70}
Rank | Item ID | Priority Score
{'-'*70}
"""
    
    for rank, (item_id, priority) in enumerate(assessment['gap_priorities'][:5], 1):
        report += f"{rank:4} | {item_id:7} | {priority:14.1f}\n"
    
    report += f"""
METADATA
{'-'*70}
Process Items Assessed:      {assessment['metadata']['process_items_count']}
Results Items Assessed:      {assessment['metadata']['results_items_count']}
Total Categories:            {assessment['metadata']['total_categories']}

{'='*70}
End of Report
{'='*70}
"""
    
    return report

# Generate report for Computer Science
report = generate_assessment_report('Computer Science', cs_assessment)
print(report)

## Export Results to JSON

In [ ]:
# Export all assessments to JSON format
export_data = {
    'assessment_date': '2024-02-01',
    'institution': 'Rajamangala University of Technology Krungthep',
    'departments': []
}

for dept, assessment in all_assessments.items():
    dept_data = {
        'department': dept,
        'organizational_score': float(assessment['organizational_score']),
        'ihi': float(assessment['ihi']),
        'maturity_level': assessment['maturity_level']['level'],
        'maturity_label': assessment['maturity_level']['label'],
        'category_scores': {k: float(v) for k, v in assessment['category_scores'].items()},
        'top_5_priorities': [
            {'item_id': item_id, 'priority_score': float(score)}
            for item_id, score in assessment['gap_priorities'][:5]
        ]
    }
    export_data['departments'].append(dept_data)

print("Assessment data exported to JSON format")
print(json.dumps(export_data, indent=2)[:500] + "...")

## Summary

This notebook demonstrated the complete organizational assessment pipeline:

1. **Data Loading**: Import assessment data from CSV
2. **Item Scoring**: Compute ADLI (process) and LeTCI (results) scores
3. **Category Aggregation**: Point-value weighted means
4. **Organizational Score**: Seven-category weighted sum
5. **Maturity Classification**: Map scores to Baldrige maturity levels
6. **Reporting**: Generate comprehensive assessment reports

### Key Outputs:
- Organizational score (0-100)
- Integration Health Index (0-1)
- Maturity level (1-5)
- Category breakdown
- Improvement priorities

## Next Steps
- **Notebook 05**: Gap prioritization with 3D matrices
- **Notebook 06**: IHI trajectory analysis